In [75]:
import os
import sys
import pandas as pd 
import numpy as np
from data_import import *
from phenotype_correlation import *
from genotype_correlation import *
from baseline_mixed_model import *
import statsmodels.api as sm

# import data
geno_df, pheno_df = import_data_svenson()

Samples:
['F326', 'F327', 'F329', 'F330', 'F331', 'F332', 'F333', 'F334', 'F335', 'F336', 'F337', 'F338', 'F339', 'F340', 'F341', 'F342', 'F343', 'F344', 'F345', 'F346', 'F347', 'F348', 'F349', 'F350', 'F351', 'F352', 'F353', 'F354', 'F355', 'F356', 'F357', 'F358', 'F359', 'F360', 'F361', 'F362', 'F363', 'F364', 'F365', 'F366', 'F367', 'F368', 'F369', 'F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'F377', 'F378', 'F379', 'F380', 'F381', 'F382', 'F383', 'F384', 'F385', 'F386', 'F387', 'F388', 'F389', 'F390', 'F391', 'F392', 'F393', 'F394', 'F395', 'F396', 'F397', 'F398', 'F399', 'F400', 'F401', 'F402', 'F403', 'F405', 'F406', 'F407', 'F408', 'F410', 'F411', 'F412', 'F413', 'F414', 'F415', 'F416', 'F417', 'F418', 'F419', 'F420', 'F421', 'F422', 'F423', 'F424', 'F425', 'M326', 'M327', 'M328', 'M329', 'M330', 'M331', 'M332', 'M333', 'M334', 'M335', 'M336', 'M337', 'M339', 'M340', 'M341', 'M342', 'M343', 'M344', 'M345', 'M347', 'M348', 'M349', 'M350', 'M351', 'M352', 'M353', 'M354', 'M355'

/Users/harryyang/Documents/Research/Class/CS229/cm-229/data_import.py:75: RuntimeWarning: Mean of empty slice
  np.nanmean(geno_df_select_transpose, axis=0))/np.nanstd(geno_df_select_transpose, axis=0)
/Users/harryyang/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [76]:
# testing with ghrelin and leptin 
phenotype_1 = "weight1"
phenotype_2 = "length1"
phenotype_3 = "fat_mri"
missing_rate = 0.5
verbose = False
sample_list = list()

In [77]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

The correlation between weight1 and length1 is 0.819920


In [78]:
for i in range(10):
    print(i)
    # separate training and test dataset 
    geno_tr, pheno_tr, geno_test, pheno_test, test_sample_list = separate_training_test(geno_select, pheno_select, missing_rate = missing_rate, sample_list_select = sample_list)

    # sklearn test 
    from sklearn.linear_model import Ridge 

    clf = Ridge(alpha = 1.0)
    a = clf.fit(y = pheno_tr[phenotype_2], X = geno_tr.transpose())

    # select top N 

    # select those genotypes only for select run
    top_N = 10
    top_N_idx = np.argsort(abs(a.coef_))[-top_N:]

    top_N_values = [a.coef_[i] for i in top_N_idx]
    

    top_N_snps = geno_tr.iloc[top_N_idx].index
    
    if verbose:
        print(top_N_snps)
        print (top_N_values)

    # perform OLS 
    lm = sm.OLS(endog = pheno_tr[phenotype_2], exog = pheno_tr[phenotype_1]).fit()

    if verbose:	
        print("The linear model summary for predicting phenotype %a based on phenotype %a" % (phenotype_2, phenotype_1))
        print(lm.summary())	
        print(lm.params)	

    # prediction for fixed effect
    predictions_fe = lm.predict(pheno_test[phenotype_1])

    # perform ridge regression on the residual (random effect part)
    residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

    # lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0)
    clf = Ridge(alpha = 1.0)
    a = clf.fit(y = residuals, X = geno_tr.loc[top_N_snps].transpose())

    # remove duplciates
    geno_test_new = geno_test.loc[:,~geno_test.columns.duplicated()]
    geno_test = geno_test_new[pheno_test[phenotype_2].index]

    prediction_re = a.predict(geno_test.loc[top_N_snps].transpose())
    total_prediction = predictions_fe + prediction_re

    new_model_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])


    # perform ridge regression on the residual (random effect part)
    residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

    lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0, alpha = 1.0)

    if verbose: 
        print(lm_re.params)

    predictions_re = lm_re.predict(geno_test.loc[top_N_snps].transpose())

    # combine the result from both
    total_prediction = predictions_fe + predictions_re

    old_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])


    print(new_model_mse, old_mse)

0
[0.0030929269467285677, -0.0031068505438668075, -0.003123072968369746, -0.0031531300597164714, -0.0031735125627936414, 0.0032554218998907813, 0.0032708802257754373, 0.0032809937859468326, -0.0033078606241894737, 0.003346251727669997]
Index(['UNC19139113', 'UNC19090096', 'UNC24326391', 'JAX00024799', 'UNC243187',
       'UNC21755855', 'UNC507751', 'JAX00071349', 'UNC19364388', 'UNC7893371'],
      dtype='object', name='marker')
2.070269203635779 2.061420258570585
1
[0.00380039084339062, -0.003893044749145584, 0.003948250543619957, 0.004101427809117771, 0.004182615712105052, 0.0044714399215011556, -0.004522147163314279, 0.004541913225294826, 0.004683499728046399, -0.005286727693305401]
Index(['UNC6015848', 'UNC9018866', 'UNC16510466', 'UNC31514370', 'JAX00118940',
       'UNC13074942', 'UNC2834974', 'UNC13078311', 'UNC13072314',
       'JAX00424241'],
      dtype='object', name='marker')
2.2819038715333018 2.386396509705147
2
[0.0035649138127884588, 0.0036786683887063826, 0.00382567299

ea